In [2]:
%pylab inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as ss
import pickle
from scipy import signal
import scipy as sc
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error as MAE
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor

Populating the interactive namespace from numpy and matplotlib


Загружаем данные о количество поездок в 102 регионах, выбранных на 2 неделе.

In [16]:
with open("./data/all_regions.pkl", "rb") as fid:
    data = pickle.load(fid)

In [17]:
with open("./data/all_regions_pass_count.pkl", "rb") as fid:
    data1 = pickle.load(fid)

In [18]:
data.transpose().shape

(21888L, 102L)

Загрузим и идентификаторы оставшихся регионов.

In [19]:
with open("./data/regions_left.pkl", "rb") as fid:
    regions = np.array(pickle.load(fid)) + 1

Создаем таблицу. По строкам у нас время, а по столбцам идентификатор ячейки.

In [20]:
p = pd.date_range('2014-01-01 00:00:00', periods=data.shape[1], freq='H')
df = pd.DataFrame(data.transpose(), index=p, columns=regions)
df.head()

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2014-01-01 00:00:00,87.0,146.0,70.0,113.0,367.0,645.0,589.0,799.0,948.0,321.0,...,9.0,0.0,5.0,89.0,10.0,35.0,9.0,106.0,22.0,71.0
2014-01-01 01:00:00,92.0,184.0,93.0,153.0,539.0,604.0,490.0,635.0,667.0,225.0,...,24.0,0.0,3.0,22.0,2.0,5.0,0.0,87.0,0.0,44.0
2014-01-01 02:00:00,108.0,165.0,55.0,151.0,443.0,571.0,465.0,499.0,455.0,124.0,...,27.0,0.0,3.0,23.0,1.0,1.0,0.0,39.0,0.0,1.0
2014-01-01 03:00:00,77.0,108.0,32.0,112.0,372.0,533.0,442.0,370.0,307.0,101.0,...,57.0,0.0,0.0,3.0,2.0,1.0,0.0,5.0,1.0,0.0
2014-01-01 04:00:00,47.0,79.0,22.0,77.0,213.0,383.0,296.0,319.0,261.0,87.0,...,38.0,0.0,1.0,9.0,1.0,8.0,0.0,29.0,1.0,18.0


In [21]:
df_pass_cnt = pd.DataFrame(data1.transpose(), index=p, columns=regions)
df_pass_cnt.head()

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2014-01-01 00:00:00,184.0,237.0,126.0,180.0,718.0,1271.0,1085.0,1458.0,1731.0,591.0,...,22.0,0.0,10.0,177.0,20.0,50.0,13.0,187.0,43.0,148.0
2014-01-01 01:00:00,167.0,334.0,173.0,297.0,1112.0,1121.0,1003.0,1184.0,1296.0,424.0,...,41.0,0.0,4.0,27.0,2.0,6.0,0.0,132.0,0.0,101.0
2014-01-01 02:00:00,210.0,305.0,91.0,295.0,818.0,1053.0,877.0,865.0,911.0,250.0,...,42.0,0.0,4.0,40.0,1.0,1.0,0.0,66.0,0.0,5.0
2014-01-01 03:00:00,148.0,175.0,54.0,209.0,662.0,963.0,829.0,689.0,570.0,181.0,...,104.0,0.0,0.0,4.0,3.0,1.0,0.0,7.0,1.0,0.0
2014-01-01 04:00:00,104.0,158.0,44.0,154.0,403.0,754.0,537.0,570.0,485.0,159.0,...,65.0,0.0,1.0,13.0,2.0,9.0,0.0,59.0,1.0,31.0


Далее привоодятся функции создающие синусы/косинусы и dummy-признаки дней недели.

In [6]:
def make_fourier_regressors(data_f, num_end, season=168.0, f_ind='week_'):
    str_var = ''
    length = data_f.shape[0]
    for i in range(1, num_end+1):
        sin = "s_" + f_ind + str(i)
        cos = "c_" + f_ind + str(i)
        data_f[sin] = np.sin(2*np.pi*i*np.arange(1, length+1)/season)
        data_f[cos] = np.cos(2*np.pi*i*np.arange(1, length+1)/season)
        str_var = str_var + sin + ' + '
        if i != num_end:
            str_var = str_var + cos + ' + '
        else:
            str_var = str_var + cos
    return str_var

def make_dummy_weekday(data):
    # Weekday dummy-variables
    data['monday'] = [1 if date.weekday() == 0 else 0 for date in data.index]
    data['tuesday'] = [1 if date.weekday() == 1 else 0 for date in data.index]
    data['wednessday'] = [1 if date.weekday() == 2 else 0 for date in data.index]
    data['thursday'] = [1 if date.weekday() == 3 else 0 for date in data.index]
    data['friday'] = [1 if date.weekday() == 4 else 0 for date in data.index]
    data['saturday'] = [1 if date.weekday() == 5 else 0 for date in data.index]
    data['sunday'] = [1 if date.weekday() == 6 else 0 for date in data.index]
    weekday_str = ' + tuesday + wednessday + thursday + friday + saturday + sunday'
    return weekday_str

def fourier_prediction(data, train_time_limit, degree=49):
    data_c = pd.DataFrame(data.values, columns = ['val'], index = data.index)
    str_reg = 'val ~ '
    week_day_str, str_var = '', ''
    str_var = make_fourier_regressors(data_c, degree)
    week_day_str = make_dummy_weekday(data_c)
    model = smf.ols(str_reg + str_var + week_day_str, data=data_c.loc[:train_time_limit])
    fitted = model.fit(cov_type='HC1')
    #regressor = ElasticNet(alpha=0.1, l1_ratio=0.6)
    #regressor.fit(data_c.loc[:train_time_limit].drop(['val'], axis=1), data_c.loc[:train_time_limit].val)
    return fitted.predict(data_c)
    #return regressor.predict(data_c.drop(['val'], axis=1))

In [7]:
R = 102
H = 739
Q_may = 0
denom = 1.0/(R*H*6)

Далее приводится функция, которая создает соответствующие выборки, обучает модели и делает предскаазания на соответствующий период времени. Используем теже признаки, что и выше, плюс скользящую сумму с окном размера 12. Параметры регрессионной модели подобраны одни и теже для всех регионов на основе качестве предсказания на майских данных.

На прошлой неделе были использованы следующие признаки: 
1. дневные и часовые лаги
2. скользящие суммы по 12 часов 
3. dummy-признаки дней недели
4. синусы/косинусы, соответствующие недельной сезонности

План на действий на данную неделю:
1. Добавить синусы/косинусы, описывающие годовую сезонность.
2. Добавить скользящих сумм.
3. Добавить dummy-признаки праздников
4. Добавить признак - температура
5. Добавить признак - температура с учетом ветра
6. Добавить признак - количество осадков за день
7. Добавить признак - средняя скорость ветра
8. Воспользоваться XGBoost регрессором, который позволяет также использовать l1 и l2 регуляризацию, но, на мой взгляд, также позволяет и учитывать некторые взаимосвязи между признаками.
9. Сабмит решения, результаты.

Посмотрим на исходное решение.

In [8]:
# Making regression for every region and than combining the error estimate for all regions
def count_error_region(data, train_time_limit, test_time_limit, region, denom, pred_start='2016-05-01', pred_end='2016-05-31', degree=49,
                       K_d=2, K_h=8):
    new_data = pd.DataFrame(data.loc[:test_time_limit][region].values, columns=['val'], \
                            index = data.loc[:test_time_limit].index)
    error = 0
    all_ids = []
    all_preds = []
    offset = max(24*K_d, K_h, 12)
    # 12-hours 
    new_data['half_day_sum'] = new_data['val'].rolling(12).sum().fillna(0)
    # fourier components
    str_var = make_fourier_regressors(new_data, degree)
    # weekday dummy components
    week_day_str = make_dummy_weekday(new_data)
    # day lags
    for day_lag in range(1, K_d):
        new_data['day_lag_'+str(day_lag)] = [0]*offset + new_data[offset-24*day_lag:-24*day_lag]['val'].values.tolist()
    # hour lags
    for hour_lag in range(1, K_h):
        new_data['hour_lag_'+str(hour_lag)] = [0]*offset + new_data[offset-hour_lag:-hour_lag]['val'].values.tolist()
    # Training and predictions
    for data_num in range(6):
        train_num_limit = data.loc[:train_time_limit].shape[0] - data_num
        regressor = ElasticNet(alpha=0.01, l1_ratio=0.4)
        regressor.fit(new_data.iloc[offset:train_num_limit].drop(['val'], axis=1), \
                      new_data.iloc[offset+data_num+1:train_num_limit+data_num+1].val)
        prediction = regressor.predict(new_data[train_time_limit:].drop(['val'], axis=1))
        difference = denom*np.abs(prediction - \
                        data.loc[pred_start+' 0'+str(0+data_num)+':00:00':pred_end+' '\
                               +str(18+data_num)+':00:00'][region].values)
        error += difference.sum()
        
        indexes = new_data[train_time_limit:].index    
        all_ids.append((pd.Series([str(region)]*indexes.size) + '_' + map(lambda x: x.strftime('%Y-%m-%d'), indexes) + '_' \
                           + map(lambda x: str(x.hour) + '_' + str(data_num+1), indexes)).values)
        all_preds.append(prediction)
    del new_data
    return error, all_preds, all_ids

Предсказание для некоторых регионов.

In [9]:
%%time
# testing elasting net parameter
err, _, _ = count_error_region(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1075, denom)
print err
err, _, _ = count_error_region(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1232, denom)
print err
err, _, _ = count_error_region(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 2168, denom)
print err

0.109867415613
0.825472819945
0.181854162141
Wall time: 35.7 s


Делаем предсказания для мая.

In [22]:
%%time
Q_may = 0
for region in df.columns:
    res, pred, ids = count_error_region(df.loc[:'2016-05-31 23:00:00'], \
                                        '2016-04-30 23:00:00', '2016-05-31 17:00:00', region, denom)
    Q_may += res
print "\n", Q_may

C:\Anaconda2\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)



20.6100811262
Wall time: 15min


Ошибка предсказания на май 2016 года. Она значительно уменьшилась по сравнению с прошлой неделей: было около 30.

Делаем предсказания на июнь по данным до мая 2016 года и считаем ошибку.

In [44]:
%%time
R = 102
H = 715
denom = 1.0/(R*H*6)
Q_june = 0
all_ids = []
all_preds = []
for region in df.columns:
    res, pred, ids = count_error_region(df.loc[:'2016-06-30 23:00:00'], \
                                        '2016-05-31 23:00:00', '2016-06-30 17:00:00', 
                                        region, denom,
                                        '2016-06-01', '2016-06-30')
    Q_june += res
    all_ids.append(ids)
    all_preds.append(pred)

20.3926429294
CPU times: user 19min 13s, sys: 2min 24s, total: 21min 37s
Wall time: 11min 54s


Добавляем dummy-признаки праздничных дней при построении моделей для каждого региона и для каждого 

In [24]:
# Making regression for every region and than combining the error estimate for all regions
def count_error_region1(data, train_time_limit, test_time_limit, region, denom, pred_start='2016-05-01', \
                        pred_end='2016-05-31', degree=49, K_d=2, K_h=8):
    new_data = pd.DataFrame(data.loc[:test_time_limit][region].values, columns=['val'], \
                            index = data.loc[:test_time_limit].index)
    error = 0
    all_ids = []
    all_preds = []
    offset = max(24*K_d, K_h, 12)
    # 12-hours 
    new_data['half_day_sum'] = new_data['val'].rolling(12).sum().fillna(0)
    # fourier components
    str_var = make_fourier_regressors(new_data, degree)
    # weekday dummy components
    week_day_str = make_dummy_weekday(new_data)
    # day lags
    for day_lag in range(1, K_d):
        new_data['day_lag_'+str(day_lag)] = [0]*offset + new_data[offset-24*day_lag:-24*day_lag]['val'].values.tolist()
    # hour lags
    for hour_lag in range(1, K_h):
        new_data['hour_lag_'+str(hour_lag)] = [0]*offset + new_data[offset-hour_lag:-hour_lag]['val'].values.tolist()
        
    # NEW FEATURES
    ##############
    # Year seasonality
    make_fourier_regressors(new_data, 6, 8766.0, 'year_')
        
    ##############
        
    # Training and predictions
    for data_num in range(6):
        train_num_limit = data.loc[:train_time_limit].shape[0] - data_num
        regressor = ElasticNet(alpha=0.01, l1_ratio=0.4)
        regressor.fit(new_data.iloc[offset:train_num_limit].drop(['val'], axis=1), \
                      new_data.iloc[offset+data_num+1:train_num_limit+data_num+1].val)
        prediction = regressor.predict(new_data[train_time_limit:].drop(['val'], axis=1))
        difference = denom*np.abs(prediction - \
                        data.loc[pred_start+' 0'+str(0+data_num)+':00:00':pred_end+' '\
                               +str(18+data_num)+':00:00'][region].values)
        error += difference.sum()
        
        indexes = new_data[train_time_limit:].index    
        all_ids.append((pd.Series([str(region)]*indexes.size) + '_' + map(lambda x: x.strftime('%Y-%m-%d'), indexes) + '_' \
                           + map(lambda x: str(x.hour) + '_' + str(data_num+1), indexes)).values)
        all_preds.append(prediction)
    del new_data
    return error, all_preds, all_ids

In [25]:
%%time
# testing elasting net parameter
err, _, _ = count_error_region1(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1075, denom)
print err
err, _, _ = count_error_region1(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1232, denom)
print err
err, _, _ = count_error_region1(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 2168, denom)
print err

0.114892784153
0.857273501547
0.181507018872
Wall time: 34.3 s


Видим, что качество ухудшилось.

In [26]:
%%time
R = 102
H = 739
Q_may = 0
denom = 1.0/(R*H*6)
Q_may = 0
for region in df.columns:
    res, pred, ids = count_error_region1(df.loc[:'2016-05-31 23:00:00'], \
                                        '2016-04-30 23:00:00', '2016-05-31 17:00:00', region, denom)
    Q_may += res
print "\n", Q_may


21.480671735
Wall time: 17min 27s


Добавление годовой сезонности заметно ухудшило модель, откажемся от их использования.

Добавим dummy-признаки праздничных дней и дней рядом с праздничными днями.

In [29]:
# Making regression for every region and than combining the error estimate for all regions
def count_error_region2(data, train_time_limit, test_time_limit, region, denom, pred_start='2016-05-01', \
                        pred_end='2016-05-31', degree=49, K_d=2, K_h=8):
    new_data = pd.DataFrame(data.loc[:test_time_limit][region].values, columns=['val'], \
                            index = data.loc[:test_time_limit].index)
    error = 0
    all_ids = []
    all_preds = []
    offset = max(24*K_d, K_h, 12)
    # 12-hours 
    new_data['half_day_sum'] = new_data['val'].rolling(12).sum().fillna(0)
    # fourier components
    str_var = make_fourier_regressors(new_data, degree)
    # weekday dummy components
    week_day_str = make_dummy_weekday(new_data)
    # day lags
    for day_lag in range(1, K_d):
        new_data['day_lag_'+str(day_lag)] = [0]*offset + new_data[offset-24*day_lag:-24*day_lag]['val'].values.tolist()
    # hour lags
    for hour_lag in range(1, K_h):
        new_data['hour_lag_'+str(hour_lag)] = [0]*offset + new_data[offset-hour_lag:-hour_lag]['val'].values.tolist()
        
    # NEW FEATURES
    ##############
    # Celebrations
    new_data['pre_new_year'] = [1 if date.month == 12 and date.day == 31 else 0 for date in new_data.index]
    new_data['new_year'] = [1 if date.month == 1 and date.day == 1 else 0 for date in new_data.index]
    new_data['ind_day'] = [1 if date.month == 7 and date.day == 4 else 0 for date in new_data.index]
    new_data['valentine_day'] = [1 if date.month == 2 and date.day == 14 else 0 for date in new_data.index]
    new_data['inag_day'] = [1 if date.month == 1 and date.day == 20 else 0 for date in new_data.index]
    new_data['veter_day'] = [1 if date.month == 11 and date.day == 11 else 0 for date in new_data.index]
    new_data['9_11_day'] = [1 if date.month == 9 and date.day == 11 else 0 for date in new_data.index]
    new_data['xmas_day'] = [1 if date.month == 12 and date.day == 25 else 0 for date in new_data.index]
    new_data['lut_day'] = [1 if date.month == 1 and ((date.year == 2014 and date.day == 20) or \
                                                     (date.year == 2015 and date.day == 19) or\
                                                     (date.year == 2016 and date.day == 18))
                            else 0 for date in new_data.index]
    new_data['labor_day'] = [1 if date.month == 9 and ((date.year == 2014 and date.day == 1) or \
                                                     (date.year == 2015 and date.day == 7) or\
                                                     (date.year == 2016 and date.day == 5))
                            else 0 for date in new_data.index]
    new_data['columb_day'] = [1 if date.month == 10 and ((date.year == 2014 and date.day == 13) or \
                                                     (date.year == 2015 and date.day == 12) or\
                                                     (date.year == 2016 and date.day == 10))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['memm_day'] = [1 if date.month == 5 and ((date.year == 2014 and date.day == 26) or \
                                                     (date.year == 2015 and date.day == 25) or\
                                                     (date.year == 2016 and date.day == 30))
                             else 0 for date in new_data.index]  #good feature
    new_data['flags_day'] = [1 if date.month == 6 and date.day == 14 else 0 for date in new_data.index]
    
        
    ##############
        
    # Training and predictions
    for data_num in range(6):
        train_num_limit = data.loc[:train_time_limit].shape[0] - data_num
        regressor = ElasticNet(alpha=0.01, l1_ratio=0.4)
        regressor.fit(new_data.iloc[offset:train_num_limit].drop(['val'], axis=1), \
                      new_data.iloc[offset+data_num+1:train_num_limit+data_num+1].val)
        prediction = regressor.predict(new_data[train_time_limit:].drop(['val'], axis=1))
        difference = denom*np.abs(prediction - \
                        data.loc[pred_start+' 0'+str(0+data_num)+':00:00':pred_end+' '\
                               +str(18+data_num)+':00:00'][region].values)
        error += difference.sum()
        
        indexes = new_data[train_time_limit:].index    
        all_ids.append((pd.Series([str(region)]*indexes.size) + '_' + map(lambda x: x.strftime('%Y-%m-%d'), indexes) + '_' \
                           + map(lambda x: str(x.hour) + '_' + str(data_num+1), indexes)).values)
        all_preds.append(prediction)
    del new_data
    return error, all_preds, all_ids

In [30]:
%%time
# testing elasting net parameter
err, _, _ = count_error_region2(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1075, denom)
print err
err, _, _ = count_error_region2(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1232, denom)
print err
err, _, _ = count_error_region2(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 2168, denom)
print err

0.109665732675
0.819150375985
0.181595258414
Wall time: 32.2 s


In [31]:
%%time
R = 102
H = 739
Q_may = 0
denom = 1.0/(R*H*6)
Q_may = 0
for region in df.columns:
    res, pred, ids = count_error_region2(df.loc[:'2016-05-31 23:00:00'], \
                                        '2016-04-30 23:00:00', '2016-05-31 17:00:00', region, denom)
    Q_may += res
print "\n", Q_may


20.5514422138
Wall time: 18min 9s


Использование праздников дало некоторый прирост качества, поэтому оставим данные признаки.

Далее будем добавлять скользящие суммы за 6, 24, 168 часов (12 было в исходной модели).

In [32]:
# Making regression for every region and than combining the error estimate for all regions
def count_error_region3(data, train_time_limit, test_time_limit, region, denom, pred_start='2016-05-01', \
                        pred_end='2016-05-31', degree=49, K_d=2, K_h=8):
    new_data = pd.DataFrame(data.loc[:test_time_limit][region].values, columns=['val'], \
                            index = data.loc[:test_time_limit].index)
    error = 0
    all_ids = []
    all_preds = []
    offset = max(24*K_d, K_h, 12)
    # 12-hours 
    new_data['half_day_sum'] = new_data['val'].rolling(12).sum().fillna(0)
    # 6-hours
    new_data['quat_day_sum'] = new_data['val'].rolling(6).sum().fillna(0)
    # 24-hours
    new_data['day_sum'] = new_data['val'].rolling(24).sum().fillna(0)
    # 168-hours
    new_data['week_sum'] = new_data['val'].rolling(168).sum().fillna(0)
    # fourier components
    str_var = make_fourier_regressors(new_data, degree)
    # weekday dummy components
    week_day_str = make_dummy_weekday(new_data)
    # day lags
    for day_lag in range(1, K_d):
        new_data['day_lag_'+str(day_lag)] = [0]*offset + new_data[offset-24*day_lag:-24*day_lag]['val'].values.tolist()
    # hour lags
    for hour_lag in range(1, K_h):
        new_data['hour_lag_'+str(hour_lag)] = [0]*offset + new_data[offset-hour_lag:-hour_lag]['val'].values.tolist()
        
    # NEW FEATURES
    ##############
    # Celebrations
    new_data['pre_new_year'] = [1 if date.month == 12 and date.day == 31 else 0 for date in new_data.index]
    new_data['new_year'] = [1 if date.month == 1 and date.day == 1 else 0 for date in new_data.index]
    new_data['ind_day'] = [1 if date.month == 7 and date.day == 4 else 0 for date in new_data.index]
    new_data['valentine_day'] = [1 if date.month == 2 and date.day == 14 else 0 for date in new_data.index]
    new_data['inag_day'] = [1 if date.month == 1 and date.day == 20 else 0 for date in new_data.index]
    new_data['veter_day'] = [1 if date.month == 11 and date.day == 11 else 0 for date in new_data.index]
    new_data['9_11_day'] = [1 if date.month == 9 and date.day == 11 else 0 for date in new_data.index]
    new_data['xmas_day'] = [1 if date.month == 12 and date.day == 25 else 0 for date in new_data.index]
    new_data['lut_day'] = [1 if date.month == 1 and ((date.year == 2014 and date.day == 20) or \
                                                     (date.year == 2015 and date.day == 19) or\
                                                     (date.year == 2016 and date.day == 18))
                            else 0 for date in new_data.index]
    new_data['labor_day'] = [1 if date.month == 9 and ((date.year == 2014 and date.day == 1) or \
                                                     (date.year == 2015 and date.day == 7) or\
                                                     (date.year == 2016 and date.day == 5))
                            else 0 for date in new_data.index]
    new_data['columb_day'] = [1 if date.month == 10 and ((date.year == 2014 and date.day == 13) or \
                                                     (date.year == 2015 and date.day == 12) or\
                                                     (date.year == 2016 and date.day == 10))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['memm_day'] = [1 if date.month == 5 and ((date.year == 2014 and date.day == 26) or \
                                                     (date.year == 2015 and date.day == 25) or\
                                                     (date.year == 2016 and date.day == 30))
                             else 0 for date in new_data.index]  #good feature
    new_data['flags_day'] = [1 if date.month == 6 and date.day == 14 else 0 for date in new_data.index]
    
        
    ##############
        
    # Training and predictions
    for data_num in range(6):
        train_num_limit = data.loc[:train_time_limit].shape[0] - data_num
        regressor = ElasticNet(alpha=0.01, l1_ratio=0.4)
        regressor.fit(new_data.iloc[offset:train_num_limit].drop(['val'], axis=1), \
                      new_data.iloc[offset+data_num+1:train_num_limit+data_num+1].val)
        prediction = regressor.predict(new_data[train_time_limit:].drop(['val'], axis=1))
        difference = denom*np.abs(prediction - \
                        data.loc[pred_start+' 0'+str(0+data_num)+':00:00':pred_end+' '\
                               +str(18+data_num)+':00:00'][region].values)
        error += difference.sum()
        
        indexes = new_data[train_time_limit:].index    
        all_ids.append((pd.Series([str(region)]*indexes.size) + '_' + map(lambda x: x.strftime('%Y-%m-%d'), indexes) + '_' \
                           + map(lambda x: str(x.hour) + '_' + str(data_num+1), indexes)).values)
        all_preds.append(prediction)
    del new_data
    return error, all_preds, all_ids

In [33]:
%%time
# testing elasting net parameter
err, _, _ = count_error_region3(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1075, denom)
print err
err, _, _ = count_error_region3(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1232, denom)
print err
err, _, _ = count_error_region3(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 2168, denom)
print err

0.108402477944
0.722976334698
0.176678274037
Wall time: 1min


In [35]:
%%time
R = 102
H = 739
Q_may = 0
denom = 1.0/(R*H*6)
Q_may = 0
for region in df.columns:
    res, pred, ids = count_error_region3(df.loc[:'2016-05-31 23:00:00'], \
                                        '2016-04-30 23:00:00', '2016-05-31 17:00:00', region, denom)
    Q_may += res
print "\n", Q_may


19.0094494726
Wall time: 35min 44s


Видим, что качество заметно улучшилось, оставим эти признаки.

Далее будем использовать почасовые температурные показатели. Эти данные были получены с помощью парсинга сайта . Код приводится в конце ноутбука.

Добавим в рассмотрение температуру окружающей среды в качестве числового признака.

In [9]:
with open("hourly_temp_data.pkl", 'rb') as fid:
    temperature = pickle.load(fid)
print "Data size: ", len(temperature)
temperature = map(float, temperature)
temperature[:5]

Data size:  21888


[-3.9, -3.9, -4.4, -4.4, -4.4]

In [10]:
# Making regression for every region and than combining the error estimate for all regions
def count_error_region4(data, train_time_limit, test_time_limit, region, denom, temperature,\
                        pred_start='2016-05-01', pred_end='2016-05-31', degree=49, K_d=2, K_h=8):
    new_data = pd.DataFrame(data.loc[:test_time_limit][region].values, columns=['val'], \
                            index = data.loc[:test_time_limit].index)
    error = 0
    all_ids = []
    all_preds = []
    offset = max(24*K_d, K_h, 12)
    # 12-hours 
    new_data['half_day_sum'] = new_data['val'].rolling(12).sum().fillna(0)
    # 6-hours
    new_data['quat_day_sum'] = new_data['val'].rolling(6).sum().fillna(0)
    # 24-hours
    new_data['day_sum'] = new_data['val'].rolling(24).sum().fillna(0)
    # 168-hours
    new_data['week_sum'] = new_data['val'].rolling(168).sum().fillna(0)
    # fourier components
    str_var = make_fourier_regressors(new_data, degree)
    # weekday dummy components
    week_day_str = make_dummy_weekday(new_data)
    # day lags
    for day_lag in range(1, K_d):
        new_data['day_lag_'+str(day_lag)] = [0]*offset + new_data[offset-24*day_lag:-24*day_lag]['val'].values.tolist()
    # hour lags
    for hour_lag in range(1, K_h):
        new_data['hour_lag_'+str(hour_lag)] = [0]*offset + new_data[offset-hour_lag:-hour_lag]['val'].values.tolist()
        
    # NEW FEATURES
    ##############
    # Celebrations
    #*************
    new_data['pre_new_year'] = [1 if date.month == 12 and date.day == 31 else 0 for date in new_data.index]
    new_data['new_year'] = [1 if date.month == 1 and date.day == 1 else 0 for date in new_data.index]
    new_data['ind_day'] = [1 if date.month == 7 and date.day == 4 else 0 for date in new_data.index]
    new_data['valentine_day'] = [1 if date.month == 2 and date.day == 14 else 0 for date in new_data.index]
    new_data['inag_day'] = [1 if date.month == 1 and date.day == 20 else 0 for date in new_data.index]
    new_data['veter_day'] = [1 if date.month == 11 and date.day == 11 else 0 for date in new_data.index]
    new_data['9_11_day'] = [1 if date.month == 9 and date.day == 11 else 0 for date in new_data.index]
    new_data['xmas_day'] = [1 if date.month == 12 and date.day == 25 else 0 for date in new_data.index]
    new_data['lut_day'] = [1 if date.month == 1 and ((date.year == 2014 and date.day == 20) or \
                                                     (date.year == 2015 and date.day == 19) or\
                                                     (date.year == 2016 and date.day == 18))
                            else 0 for date in new_data.index]
    new_data['labor_day'] = [1 if date.month == 9 and ((date.year == 2014 and date.day == 1) or \
                                                     (date.year == 2015 and date.day == 7) or\
                                                     (date.year == 2016 and date.day == 5))
                            else 0 for date in new_data.index]
    new_data['columb_day'] = [1 if date.month == 10 and ((date.year == 2014 and date.day == 13) or \
                                                     (date.year == 2015 and date.day == 12) or\
                                                     (date.year == 2016 and date.day == 10))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['memm_day'] = [1 if date.month == 5 and ((date.year == 2014 and date.day == 26) or \
                                                     (date.year == 2015 and date.day == 25) or\
                                                     (date.year == 2016 and date.day == 30))
                             else 0 for date in new_data.index]  #good feature
    new_data['flags_day'] = [1 if date.month == 6 and date.day == 14 else 0 for date in new_data.index]
    #********
    # Temperature
    new_data['temperature'] = temperature[:new_data.shape[0]]
        
    ##############
        
    # Training and predictions
    for data_num in range(6):
        train_num_limit = data.loc[:train_time_limit].shape[0] - data_num
        regressor = ElasticNet(alpha=0.01, l1_ratio=0.4, max_iter=1500)
        regressor.fit(new_data.iloc[offset:train_num_limit].drop(['val'], axis=1), \
                      new_data.iloc[offset+data_num+1:train_num_limit+data_num+1].val)
        prediction = regressor.predict(new_data[train_time_limit:].drop(['val'], axis=1))
        difference = denom*np.abs(prediction - \
                        data.loc[pred_start+' 0'+str(0+data_num)+':00:00':pred_end+' '\
                               +str(18+data_num)+':00:00'][region].values)
        error += difference.sum()
        
        indexes = new_data[train_time_limit:].index    
        all_ids.append((pd.Series([str(region)]*indexes.size) + '_' + map(lambda x: x.strftime('%Y-%m-%d'), indexes) + '_' \
                           + map(lambda x: str(x.hour) + '_' + str(data_num+1), indexes)).values)
        all_preds.append(prediction)
    del new_data
    return error, all_preds, all_ids

In [14]:
%%time
# testing elasting net parameter
err, _, _ = count_error_region4(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1075,\
                                denom, temperature)
print err
err, _, _ = count_error_region4(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1232,\
                                denom, temperature)
print err
err, _, _ = count_error_region4(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 2168,\
                                denom, temperature)
print err

0.108018391827
0.721881773709
0.176647576885
Wall time: 1min 34s


In [18]:
%%time
R = 102
H = 739
Q_may = 0
denom = 1.0/(R*H*6)
Q_may = 0
for region in df.columns:
    res, pred, ids = count_error_region4(df.loc[:'2016-05-31 23:00:00'], \
                                        '2016-04-30 23:00:00', '2016-05-31 17:00:00', region, denom, temperature)
    Q_may += res
print "\n", Q_may


19.0005933588
Wall time: 1h 1min 56s


Качество немного лучшилось.

Добавляем признак - температура с учетом ветра.

In [19]:
with open("hourly_temp_wind_data.pkl", 'rb') as fid:
    temperature_wind = pickle.load(fid)
print "Data size: ", len(temperature_wind)
temperature_wind = map(float, temperature_wind)
temperature_wind[:5]

Data size:  21888


[-6.3, -9.2, -9.8, -9.8, -13.9]

In [20]:
# Making regression for every region and than combining the error estimate for all regions
def count_error_region5(data, train_time_limit, test_time_limit, region, denom, temperature, temperature_wind,\
                        pred_start='2016-05-01', pred_end='2016-05-31', degree=49, K_d=2, K_h=8):
    new_data = pd.DataFrame(data.loc[:test_time_limit][region].values, columns=['val'], \
                            index = data.loc[:test_time_limit].index)
    error = 0
    all_ids = []
    all_preds = []
    offset = max(24*K_d, K_h, 12)
    # 12-hours 
    new_data['half_day_sum'] = new_data['val'].rolling(12).sum().fillna(0)
    # 6-hours
    new_data['quat_day_sum'] = new_data['val'].rolling(6).sum().fillna(0)
    # 24-hours
    new_data['day_sum'] = new_data['val'].rolling(24).sum().fillna(0)
    # 168-hours
    new_data['week_sum'] = new_data['val'].rolling(168).sum().fillna(0)
    # fourier components
    str_var = make_fourier_regressors(new_data, degree)
    # weekday dummy components
    week_day_str = make_dummy_weekday(new_data)
    # day lags
    for day_lag in range(1, K_d):
        new_data['day_lag_'+str(day_lag)] = [0]*offset + new_data[offset-24*day_lag:-24*day_lag]['val'].values.tolist()
    # hour lags
    for hour_lag in range(1, K_h):
        new_data['hour_lag_'+str(hour_lag)] = [0]*offset + new_data[offset-hour_lag:-hour_lag]['val'].values.tolist()
        
    # NEW FEATURES
    ##############
    # Celebrations
    #*************
    new_data['pre_new_year'] = [1 if date.month == 12 and date.day == 31 else 0 for date in new_data.index]
    new_data['new_year'] = [1 if date.month == 1 and date.day == 1 else 0 for date in new_data.index]
    new_data['ind_day'] = [1 if date.month == 7 and date.day == 4 else 0 for date in new_data.index]
    new_data['valentine_day'] = [1 if date.month == 2 and date.day == 14 else 0 for date in new_data.index]
    new_data['inag_day'] = [1 if date.month == 1 and date.day == 20 else 0 for date in new_data.index]
    new_data['veter_day'] = [1 if date.month == 11 and date.day == 11 else 0 for date in new_data.index]
    new_data['9_11_day'] = [1 if date.month == 9 and date.day == 11 else 0 for date in new_data.index]
    new_data['xmas_day'] = [1 if date.month == 12 and date.day == 25 else 0 for date in new_data.index]
    new_data['lut_day'] = [1 if date.month == 1 and ((date.year == 2014 and date.day == 20) or \
                                                     (date.year == 2015 and date.day == 19) or\
                                                     (date.year == 2016 and date.day == 18))
                            else 0 for date in new_data.index]
    new_data['labor_day'] = [1 if date.month == 9 and ((date.year == 2014 and date.day == 1) or \
                                                     (date.year == 2015 and date.day == 7) or\
                                                     (date.year == 2016 and date.day == 5))
                            else 0 for date in new_data.index]
    new_data['columb_day'] = [1 if date.month == 10 and ((date.year == 2014 and date.day == 13) or \
                                                     (date.year == 2015 and date.day == 12) or\
                                                     (date.year == 2016 and date.day == 10))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['memm_day'] = [1 if date.month == 5 and ((date.year == 2014 and date.day == 26) or \
                                                     (date.year == 2015 and date.day == 25) or\
                                                     (date.year == 2016 and date.day == 30))
                             else 0 for date in new_data.index]  #good feature
    new_data['flags_day'] = [1 if date.month == 6 and date.day == 14 else 0 for date in new_data.index]
    #********
    # Temperature
    new_data['temperature'] = temperature[:new_data.shape[0]]
    new_data['temperature_wind'] = temperature[:new_data.shape[0]]
        
    ##############
        
    # Training and predictions
    for data_num in range(6):
        train_num_limit = data.loc[:train_time_limit].shape[0] - data_num
        regressor = ElasticNet(alpha=0.01, l1_ratio=0.4)
        regressor.fit(new_data.iloc[offset:train_num_limit].drop(['val'], axis=1), \
                      new_data.iloc[offset+data_num+1:train_num_limit+data_num+1].val)
        prediction = regressor.predict(new_data[train_time_limit:].drop(['val'], axis=1))
        difference = denom*np.abs(prediction - \
                        data.loc[pred_start+' 0'+str(0+data_num)+':00:00':pred_end+' '\
                               +str(18+data_num)+':00:00'][region].values)
        error += difference.sum()
        
        indexes = new_data[train_time_limit:].index    
        all_ids.append((pd.Series([str(region)]*indexes.size) + '_' + map(lambda x: x.strftime('%Y-%m-%d'), indexes) + '_' \
                           + map(lambda x: str(x.hour) + '_' + str(data_num+1), indexes)).values)
        all_preds.append(prediction)
    del new_data
    return error, all_preds, all_ids

In [21]:
%%time
# testing elasting net parameter
err, _, _ = count_error_region5(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1075,\
                                denom, temperature, temperature_wind)
print err
err, _, _ = count_error_region5(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1232,\
                                denom, temperature, temperature_wind)
print err
err, _, _ = count_error_region5(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 2168,\
                                denom, temperature, temperature_wind)
print err

0.108014167034
0.721861187392
0.176647575811
Wall time: 1min 2s


In [22]:
%%time
R = 102
H = 739
Q_may = 0
denom = 1.0/(R*H*6)
Q_may = 0
for region in df.columns:
    res, pred, ids = count_error_region5(df.loc[:'2016-05-31 23:00:00'], \
                                        '2016-04-30 23:00:00', '2016-05-31 17:00:00', region,\
                                         denom, temperature, temperature_wind)
    Q_may += res
print "\n", Q_may


18.9890127521
Wall time: 33min 50s


Качество еще улучшилось.

Другие признаки: количество осадков за день, скорость ветра, потом введем dummy-признаки погодных событий: туман, снег, дождь. (Используем для всех часов в один день одни и теже значения).

In [23]:
daily_weather = pd.read_csv("./DailyWeather.csv", sep='\t')
daily_weather.fillna('NA', inplace=True)
daily_weather.replace('T', 0, inplace=True)
daily_weather.head()

,Day,Temp high,Temp avg,Temp low,Dew high,Dew avg,Dew low,Hum high,Hum avg,Hum low,...,Press avg,Press low,Vis high,Vis avg,Vis low,Wind high,Wind avg,Wind high.1,Precip,Events
0,1,1,-2,-4,-7,-11,-15,59,49,38,...,1027,1025,16,16,16,23,9,37,0.00,NA
1,2,1,-3,-8,-5,-7,-10,84,70,56,...,1015,1005,16,12,1,34,20,45,8.38,"Mist , Snow"
2,3,-8,-10,-13,-11,-17,-22,84,63,42,...,1018,1004,16,9,0,34,17,47,7.37,Snow
3,4,-2,-7,-13,-9,-16,-21,66,51,35,...,1030,1026,16,16,16,14,8,32,0.00,NA
4,5,4,1,-3,3,-3,-8,92,75,58,...,1023,1010,16,5,0,11,6,27,3.56,"Mist , Rain"


In [24]:
daily_weather.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 21 columns):
Day            912 non-null int64
Temp high      912 non-null int64
Temp avg       912 non-null int64
Temp low       912 non-null int64
Dew high       912 non-null int64
Dew avg        912 non-null int64
Dew low        912 non-null int64
Hum high       912 non-null int64
Hum avg        912 non-null int64
Hum low        912 non-null int64
Press high     912 non-null object
Press avg      912 non-null object
Press low      912 non-null object
Vis high       912 non-null object
Vis avg        912 non-null object
Vis low        912 non-null object
Wind high      912 non-null object
Wind avg       912 non-null object
Wind high.1    912 non-null object
Precip         912 non-null object
Events         912 non-null object
dtypes: int64(10), object(11)
memory usage: 149.7+ KB


In [25]:
daily_weather_h = []
for ind in daily_weather.index:
    for j in range(24):
        daily_weather_h.append(daily_weather.iloc[ind].tolist())
daily_weather_h_df = pd.DataFrame(daily_weather_h, columns = daily_weather.columns)
daily_weather_h_df.head()

,Day,Temp high,Temp avg,Temp low,Dew high,Dew avg,Dew low,Hum high,Hum avg,Hum low,...,Press avg,Press low,Vis high,Vis avg,Vis low,Wind high,Wind avg,Wind high.1,Precip,Events
0,1,1,-2,-4,-7,-11,-15,59,49,38,...,1027,1025,16,16,16,23,9,37,0.00,NA
1,1,1,-2,-4,-7,-11,-15,59,49,38,...,1027,1025,16,16,16,23,9,37,0.00,NA
2,1,1,-2,-4,-7,-11,-15,59,49,38,...,1027,1025,16,16,16,23,9,37,0.00,NA
3,1,1,-2,-4,-7,-11,-15,59,49,38,...,1027,1025,16,16,16,23,9,37,0.00,NA
4,1,1,-2,-4,-7,-11,-15,59,49,38,...,1027,1025,16,16,16,23,9,37,0.00,NA


In [26]:
daily_weather_h_df.Precip = daily_weather_h_df.Precip.apply(float)
daily_weather_h_df['Wind avg'].replace('-', 0, inplace=True)
daily_weather_h_df['Wind avg'] = daily_weather_h_df['Wind avg'].apply(float)

In [27]:
daily_weather_h_df.shape

(21888, 21)

Добавляем количество осадков.

In [28]:
# Making regression for every region and than combining the error estimate for all regions
def count_error_region6(data, train_time_limit, test_time_limit, region, denom, temperature, temperature_wind,\
                        dailyWeather, pred_start='2016-05-01', pred_end='2016-05-31', degree=49, K_d=2, K_h=8):
    new_data = pd.DataFrame(data.loc[:test_time_limit][region].values, columns=['val'], \
                            index = data.loc[:test_time_limit].index)
    error = 0
    all_ids = []
    all_preds = []
    offset = max(24*K_d, K_h, 12)
    # 12-hours 
    new_data['half_day_sum'] = new_data['val'].rolling(12).sum().fillna(0)
    # 6-hours
    new_data['quat_day_sum'] = new_data['val'].rolling(6).sum().fillna(0)
    # 24-hours
    new_data['day_sum'] = new_data['val'].rolling(24).sum().fillna(0)
    # 168-hours
    new_data['week_sum'] = new_data['val'].rolling(168).sum().fillna(0)
    # fourier components
    str_var = make_fourier_regressors(new_data, degree)
    # weekday dummy components
    week_day_str = make_dummy_weekday(new_data)
    # day lags
    for day_lag in range(1, K_d):
        new_data['day_lag_'+str(day_lag)] = [0]*offset + new_data[offset-24*day_lag:-24*day_lag]['val'].values.tolist()
    # hour lags
    for hour_lag in range(1, K_h):
        new_data['hour_lag_'+str(hour_lag)] = [0]*offset + new_data[offset-hour_lag:-hour_lag]['val'].values.tolist()
        
    # NEW FEATURES
    ##############
    # Celebrations
    #*************
    new_data['pre_new_year'] = [1 if date.month == 12 and date.day == 31 else 0 for date in new_data.index]
    new_data['new_year'] = [1 if date.month == 1 and date.day == 1 else 0 for date in new_data.index]
    new_data['ind_day'] = [1 if date.month == 7 and date.day == 4 else 0 for date in new_data.index]
    new_data['valentine_day'] = [1 if date.month == 2 and date.day == 14 else 0 for date in new_data.index]
    new_data['inag_day'] = [1 if date.month == 1 and date.day == 20 else 0 for date in new_data.index]
    new_data['veter_day'] = [1 if date.month == 11 and date.day == 11 else 0 for date in new_data.index]
    new_data['9_11_day'] = [1 if date.month == 9 and date.day == 11 else 0 for date in new_data.index]
    new_data['xmas_day'] = [1 if date.month == 12 and date.day == 25 else 0 for date in new_data.index]
    new_data['lut_day'] = [1 if date.month == 1 and ((date.year == 2014 and date.day == 20) or \
                                                     (date.year == 2015 and date.day == 19) or\
                                                     (date.year == 2016 and date.day == 18))
                            else 0 for date in new_data.index]
    new_data['labor_day'] = [1 if date.month == 9 and ((date.year == 2014 and date.day == 1) or \
                                                     (date.year == 2015 and date.day == 7) or\
                                                     (date.year == 2016 and date.day == 5))
                            else 0 for date in new_data.index]
    new_data['columb_day'] = [1 if date.month == 10 and ((date.year == 2014 and date.day == 13) or \
                                                     (date.year == 2015 and date.day == 12) or\
                                                     (date.year == 2016 and date.day == 10))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['memm_day'] = [1 if date.month == 5 and ((date.year == 2014 and date.day == 26) or \
                                                     (date.year == 2015 and date.day == 25) or\
                                                     (date.year == 2016 and date.day == 30))
                             else 0 for date in new_data.index]  #good feature
    new_data['flags_day'] = [1 if date.month == 6 and date.day == 14 else 0 for date in new_data.index]
    #********
    # Temperature
    new_data['temperature'] = temperature[:new_data.shape[0]]
    new_data['temperature_wind'] = temperature[:new_data.shape[0]]
    
    # Daily precipitation
    new_data['precipitation'] = dailyWeather['Precip'].values[:new_data.shape[0]]
        
    ##############
        
    # Training and predictions
    for data_num in range(6):
        train_num_limit = data.loc[:train_time_limit].shape[0] - data_num
        regressor = ElasticNet(alpha=0.01, l1_ratio=0.4)
        regressor.fit(new_data.iloc[offset:train_num_limit].drop(['val'], axis=1), \
                      new_data.iloc[offset+data_num+1:train_num_limit+data_num+1].val)
        prediction = regressor.predict(new_data[train_time_limit:].drop(['val'], axis=1))
        difference = denom*np.abs(prediction - \
                        data.loc[pred_start+' 0'+str(0+data_num)+':00:00':pred_end+' '\
                               +str(18+data_num)+':00:00'][region].values)
        error += difference.sum()
        
        indexes = new_data[train_time_limit:].index    
        all_ids.append((pd.Series([str(region)]*indexes.size) + '_' + map(lambda x: x.strftime('%Y-%m-%d'), indexes) + '_' \
                           + map(lambda x: str(x.hour) + '_' + str(data_num+1), indexes)).values)
        all_preds.append(prediction)
    del new_data
    return error, all_preds, all_ids

In [29]:
%%time
# testing elasting net parameter
err, _, _ = count_error_region6(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1075,\
                                denom, temperature, temperature_wind, daily_weather_h_df)
print err
err, _, _ = count_error_region6(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1232,\
                                denom, temperature, temperature_wind, daily_weather_h_df)
print err
err, _, _ = count_error_region6(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 2168,\
                                denom, temperature, temperature_wind, daily_weather_h_df)
print err

0.106966956187
0.72018189442
0.176846396283
Wall time: 1min 3s


In [30]:
%%time
R = 102
H = 739
Q_may = 0
denom = 1.0/(R*H*6)
Q_may = 0
for region in df.columns:
    res, pred, ids = count_error_region6(df.loc[:'2016-05-31 23:00:00'], \
                                        '2016-04-30 23:00:00', '2016-05-31 17:00:00', region,\
                                         denom, temperature, temperature_wind, daily_weather_h_df)
    Q_may += res
print "\n", Q_may


18.9588077377
Wall time: 36min 20s


Качество еще немного улучшилось - оставляем данный признак. Добавим в рассмотрение среднюю скорость ветра.

In [31]:
# Making regression for every region and than combining the error estimate for all regions
def count_error_region7(data, train_time_limit, test_time_limit, region, denom, temperature, temperature_wind,\
                        dailyWeather, pred_start='2016-05-01', pred_end='2016-05-31', degree=49, K_d=2, K_h=8):
    new_data = pd.DataFrame(data.loc[:test_time_limit][region].values, columns=['val'], \
                            index = data.loc[:test_time_limit].index)
    error = 0
    all_ids = []
    all_preds = []
    offset = max(24*K_d, K_h, 12)
    # 12-hours 
    new_data['half_day_sum'] = new_data['val'].rolling(12).sum().fillna(0)
    # 6-hours
    new_data['quat_day_sum'] = new_data['val'].rolling(6).sum().fillna(0)
    # 24-hours
    new_data['day_sum'] = new_data['val'].rolling(24).sum().fillna(0)
    # 168-hours
    new_data['week_sum'] = new_data['val'].rolling(168).sum().fillna(0)
    # fourier components
    str_var = make_fourier_regressors(new_data, degree)
    # weekday dummy components
    week_day_str = make_dummy_weekday(new_data)
    # day lags
    for day_lag in range(1, K_d):
        new_data['day_lag_'+str(day_lag)] = [0]*offset + new_data[offset-24*day_lag:-24*day_lag]['val'].values.tolist()
    # hour lags
    for hour_lag in range(1, K_h):
        new_data['hour_lag_'+str(hour_lag)] = [0]*offset + new_data[offset-hour_lag:-hour_lag]['val'].values.tolist()
        
    # NEW FEATURES
    ##############
    # Celebrations
    #*************
    new_data['pre_new_year'] = [1 if date.month == 12 and date.day == 31 else 0 for date in new_data.index]
    new_data['new_year'] = [1 if date.month == 1 and date.day == 1 else 0 for date in new_data.index]
    new_data['ind_day'] = [1 if date.month == 7 and date.day == 4 else 0 for date in new_data.index]
    new_data['valentine_day'] = [1 if date.month == 2 and date.day == 14 else 0 for date in new_data.index]
    new_data['inag_day'] = [1 if date.month == 1 and date.day == 20 else 0 for date in new_data.index]
    new_data['veter_day'] = [1 if date.month == 11 and date.day == 11 else 0 for date in new_data.index]
    new_data['9_11_day'] = [1 if date.month == 9 and date.day == 11 else 0 for date in new_data.index]
    new_data['xmas_day'] = [1 if date.month == 12 and date.day == 25 else 0 for date in new_data.index]
    new_data['lut_day'] = [1 if date.month == 1 and ((date.year == 2014 and date.day == 20) or \
                                                     (date.year == 2015 and date.day == 19) or\
                                                     (date.year == 2016 and date.day == 18))
                            else 0 for date in new_data.index]
    new_data['labor_day'] = [1 if date.month == 9 and ((date.year == 2014 and date.day == 1) or \
                                                     (date.year == 2015 and date.day == 7) or\
                                                     (date.year == 2016 and date.day == 5))
                            else 0 for date in new_data.index]
    new_data['columb_day'] = [1 if date.month == 10 and ((date.year == 2014 and date.day == 13) or \
                                                     (date.year == 2015 and date.day == 12) or\
                                                     (date.year == 2016 and date.day == 10))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['memm_day'] = [1 if date.month == 5 and ((date.year == 2014 and date.day == 26) or \
                                                     (date.year == 2015 and date.day == 25) or\
                                                     (date.year == 2016 and date.day == 30))
                             else 0 for date in new_data.index]  #good feature
    new_data['flags_day'] = [1 if date.month == 6 and date.day == 14 else 0 for date in new_data.index]
    #********
    # Temperature
    new_data['temperature'] = temperature[:new_data.shape[0]]
    new_data['temperature_wind'] = temperature[:new_data.shape[0]]
    
    # Daily precipitation
    new_data['precipitation'] = dailyWeather['Precip'].values[:new_data.shape[0]]
    new_data['wind_avg'] = daily_weather_h_df['Wind avg'].values[:new_data.shape[0]]
        
    ##############
        
    # Training and predictions
    for data_num in range(6):
        train_num_limit = data.loc[:train_time_limit].shape[0] - data_num
        regressor = ElasticNet(alpha=0.01, l1_ratio=0.4)
        regressor.fit(new_data.iloc[offset:train_num_limit].drop(['val'], axis=1), \
                      new_data.iloc[offset+data_num+1:train_num_limit+data_num+1].val)
        prediction = regressor.predict(new_data[train_time_limit:].drop(['val'], axis=1))
        difference = denom*np.abs(prediction - \
                        data.loc[pred_start+' 0'+str(0+data_num)+':00:00':pred_end+' '\
                               +str(18+data_num)+':00:00'][region].values)
        error += difference.sum()
        
        indexes = new_data[train_time_limit:].index    
        all_ids.append((pd.Series([str(region)]*indexes.size) + '_' + map(lambda x: x.strftime('%Y-%m-%d'), indexes) + '_' \
                           + map(lambda x: str(x.hour) + '_' + str(data_num+1), indexes)).values)
        all_preds.append(prediction)
    del new_data
    return error, all_preds, all_ids

In [32]:
%%time
# testing elasting net parameter
err, _, _ = count_error_region7(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1075,\
                                denom, temperature, temperature_wind, daily_weather_h_df)
print err
err, _, _ = count_error_region7(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1232,\
                                denom, temperature, temperature_wind, daily_weather_h_df)
print err
err, _, _ = count_error_region7(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 2168,\
                                denom, temperature, temperature_wind, daily_weather_h_df)
print err

0.106993174123
0.719541178783
0.176854346763
Wall time: 1min 12s


In [33]:
%%time
R = 102
H = 739
Q_may = 0
denom = 1.0/(R*H*6)
Q_may = 0
for region in df.columns:
    res, pred, ids = count_error_region7(df.loc[:'2016-05-31 23:00:00'], \
                                        '2016-04-30 23:00:00', '2016-05-31 17:00:00', region,\
                                         denom, temperature, temperature_wind, daily_weather_h_df)
    Q_may += res
print "\n", Q_may


18.9590542188
Wall time: 36min 20s


Заметно улучшения качества не получили. Добавим dummy-признаки погодных событий: снег, дождь, туман.

In [34]:
# Making regression for every region and than combining the error estimate for all regions
def count_error_region8(data, train_time_limit, test_time_limit, region, denom, temperature, temperature_wind,\
                        dailyWeather, pred_start='2016-05-01', pred_end='2016-05-31', degree=49, K_d=2, K_h=8):
    new_data = pd.DataFrame(data.loc[:test_time_limit][region].values, columns=['val'], \
                            index = data.loc[:test_time_limit].index)
    error = 0
    all_ids = []
    all_preds = []
    offset = max(24*K_d, K_h, 12)
    # 12-hours 
    new_data['half_day_sum'] = new_data['val'].rolling(12).sum().fillna(0)
    # 6-hours
    new_data['quat_day_sum'] = new_data['val'].rolling(6).sum().fillna(0)
    # 24-hours
    new_data['day_sum'] = new_data['val'].rolling(24).sum().fillna(0)
    # 168-hours
    new_data['week_sum'] = new_data['val'].rolling(168).sum().fillna(0)
    # fourier components
    str_var = make_fourier_regressors(new_data, degree)
    # weekday dummy components
    week_day_str = make_dummy_weekday(new_data)
    # day lags
    for day_lag in range(1, K_d):
        new_data['day_lag_'+str(day_lag)] = [0]*offset + new_data[offset-24*day_lag:-24*day_lag]['val'].values.tolist()
    # hour lags
    for hour_lag in range(1, K_h):
        new_data['hour_lag_'+str(hour_lag)] = [0]*offset + new_data[offset-hour_lag:-hour_lag]['val'].values.tolist()
        
    # NEW FEATURES
    ##############
    # Celebrations
    #*************
    new_data['pre_new_year'] = [1 if date.month == 12 and date.day == 31 else 0 for date in new_data.index]
    new_data['new_year'] = [1 if date.month == 1 and date.day == 1 else 0 for date in new_data.index]
    new_data['ind_day'] = [1 if date.month == 7 and date.day == 4 else 0 for date in new_data.index]
    new_data['valentine_day'] = [1 if date.month == 2 and date.day == 14 else 0 for date in new_data.index]
    new_data['inag_day'] = [1 if date.month == 1 and date.day == 20 else 0 for date in new_data.index]
    new_data['veter_day'] = [1 if date.month == 11 and date.day == 11 else 0 for date in new_data.index]
    new_data['9_11_day'] = [1 if date.month == 9 and date.day == 11 else 0 for date in new_data.index]
    new_data['xmas_day'] = [1 if date.month == 12 and date.day == 25 else 0 for date in new_data.index]
    new_data['lut_day'] = [1 if date.month == 1 and ((date.year == 2014 and date.day == 20) or \
                                                     (date.year == 2015 and date.day == 19) or\
                                                     (date.year == 2016 and date.day == 18))
                            else 0 for date in new_data.index]
    new_data['labor_day'] = [1 if date.month == 9 and ((date.year == 2014 and date.day == 1) or \
                                                     (date.year == 2015 and date.day == 7) or\
                                                     (date.year == 2016 and date.day == 5))
                            else 0 for date in new_data.index]
    new_data['columb_day'] = [1 if date.month == 10 and ((date.year == 2014 and date.day == 13) or \
                                                     (date.year == 2015 and date.day == 12) or\
                                                     (date.year == 2016 and date.day == 10))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['memm_day'] = [1 if date.month == 5 and ((date.year == 2014 and date.day == 26) or \
                                                     (date.year == 2015 and date.day == 25) or\
                                                     (date.year == 2016 and date.day == 30))
                             else 0 for date in new_data.index]  #good feature
    new_data['flags_day'] = [1 if date.month == 6 and date.day == 14 else 0 for date in new_data.index]
    #********
    # Temperature
    new_data['temperature'] = temperature[:new_data.shape[0]]
    new_data['temperature_wind'] = temperature[:new_data.shape[0]]
    
    # Daily precipitation
    new_data['precipitation'] = dailyWeather['Precip'].values[:new_data.shape[0]]
    new_data['wind_avg'] = daily_weather_h_df['Wind avg'].values[:new_data.shape[0]]
    new_data['mist'] = [1 if 'Mist' in string else 0 for string in dailyWeather.Events.values[:new_data.shape[0]]]
    new_data['snow'] = [1 if 'Snow' in string else 0 for string in dailyWeather.Events.values[:new_data.shape[0]]]
    new_data['rain'] = [1 if 'Rain' in string else 0 for string in dailyWeather.Events.values[:new_data.shape[0]]]
        
    ##############
        
    # Training and predictions
    for data_num in range(6):
        train_num_limit = data.loc[:train_time_limit].shape[0] - data_num
        regressor = ElasticNet(alpha=0.01, l1_ratio=0.4)
        regressor.fit(new_data.iloc[offset:train_num_limit].drop(['val'], axis=1), \
                      new_data.iloc[offset+data_num+1:train_num_limit+data_num+1].val)
        prediction = regressor.predict(new_data[train_time_limit:].drop(['val'], axis=1))
        difference = denom*np.abs(prediction - \
                        data.loc[pred_start+' 0'+str(0+data_num)+':00:00':pred_end+' '\
                               +str(18+data_num)+':00:00'][region].values)
        error += difference.sum()
        
        indexes = new_data[train_time_limit:].index    
        all_ids.append((pd.Series([str(region)]*indexes.size) + '_' + map(lambda x: x.strftime('%Y-%m-%d'), indexes) + '_' \
                           + map(lambda x: str(x.hour) + '_' + str(data_num+1), indexes)).values)
        all_preds.append(prediction)
    del new_data
    return error, all_preds, all_ids

In [35]:
%%time
# testing elasting net parameter
err, _, _ = count_error_region8(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1075,\
                                denom, temperature, temperature_wind, daily_weather_h_df)
print err
err, _, _ = count_error_region8(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1232,\
                                denom, temperature, temperature_wind, daily_weather_h_df)
print err
err, _, _ = count_error_region8(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 2168,\
                                denom, temperature, temperature_wind, daily_weather_h_df)
print err

0.107092081043
0.721483936195
0.17688974258
Wall time: 1min 7s


In [37]:
%%time
R = 102
H = 739
Q_may = 0
denom = 1.0/(R*H*6)
Q_may = 0
for region in df.columns:
    res, pred, ids = count_error_region8(df.loc[:'2016-05-31 23:00:00'], \
                                        '2016-04-30 23:00:00', '2016-05-31 17:00:00', region,\
                                         denom, temperature, temperature_wind, daily_weather_h_df)
    Q_may += res
print "\n", Q_may


18.9903296916
Wall time: 41min 9s


Качество модели ухудшилось, уберем данные признаки из модели.

Можно попробовать добавлять еще разных параметров в модель, но остановимся на варианте 7. Далее попробуем применить другой алгоритм обучения: XGBoostRegressor.

In [39]:
import xgboost as xgb

In [65]:
# Making regression for every region and than combining the error estimate for all regions
def count_error_regionGDB(data, train_time_limit, test_time_limit, region, denom, temperature, temperature_wind,\
                        dailyWeather, pred_start='2016-05-01', pred_end='2016-05-31', degree=49, K_d=2, K_h=8):
    new_data = pd.DataFrame(data.loc[:test_time_limit][region].values, columns=['val'], \
                            index = data.loc[:test_time_limit].index)
    error = 0
    all_ids = []
    all_preds = []
    offset = max(24*K_d, K_h, 12)
    # 12-hours 
    new_data['half_day_sum'] = new_data['val'].rolling(12).sum().fillna(0)
    # 6-hours
    new_data['quat_day_sum'] = new_data['val'].rolling(6).sum().fillna(0)
    # 24-hours
    new_data['day_sum'] = new_data['val'].rolling(24).sum().fillna(0)
    # 168-hours
    new_data['week_sum'] = new_data['val'].rolling(168).sum().fillna(0)
    # fourier components
    str_var = make_fourier_regressors(new_data, degree)
    # weekday dummy components
    week_day_str = make_dummy_weekday(new_data)
    # day lags
    for day_lag in range(1, K_d):
        new_data['day_lag_'+str(day_lag)] = [0]*offset + new_data[offset-24*day_lag:-24*day_lag]['val'].values.tolist()
    # hour lags
    for hour_lag in range(1, K_h):
        new_data['hour_lag_'+str(hour_lag)] = [0]*offset + new_data[offset-hour_lag:-hour_lag]['val'].values.tolist()
        
    # NEW FEATURES
    ##############
    # Celebrations
    #*************
    new_data['pre_new_year'] = [1 if date.month == 12 and date.day == 31 else 0 for date in new_data.index]
    new_data['new_year'] = [1 if date.month == 1 and date.day == 1 else 0 for date in new_data.index]
    new_data['ind_day'] = [1 if date.month == 7 and date.day == 4 else 0 for date in new_data.index]
    new_data['valentine_day'] = [1 if date.month == 2 and date.day == 14 else 0 for date in new_data.index]
    new_data['inag_day'] = [1 if date.month == 1 and date.day == 20 else 0 for date in new_data.index]
    new_data['veter_day'] = [1 if date.month == 11 and date.day == 11 else 0 for date in new_data.index]
    new_data['9_11_day'] = [1 if date.month == 9 and date.day == 11 else 0 for date in new_data.index]
    new_data['xmas_day'] = [1 if date.month == 12 and date.day == 25 else 0 for date in new_data.index]
    new_data['lut_day'] = [1 if date.month == 1 and ((date.year == 2014 and date.day == 20) or \
                                                     (date.year == 2015 and date.day == 19) or\
                                                     (date.year == 2016 and date.day == 18))
                            else 0 for date in new_data.index]
    new_data['labor_day'] = [1 if date.month == 9 and ((date.year == 2014 and date.day == 1) or \
                                                     (date.year == 2015 and date.day == 7) or\
                                                     (date.year == 2016 and date.day == 5))
                            else 0 for date in new_data.index]
    new_data['columb_day'] = [1 if date.month == 10 and ((date.year == 2014 and date.day == 13) or \
                                                     (date.year == 2015 and date.day == 12) or\
                                                     (date.year == 2016 and date.day == 10))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                     (date.year == 2015 and date.day == 16) or\
                                                     (date.year == 2016 and date.day == 15))
                             else 0 for date in new_data.index]
    new_data['memm_day'] = [1 if date.month == 5 and ((date.year == 2014 and date.day == 26) or \
                                                     (date.year == 2015 and date.day == 25) or\
                                                     (date.year == 2016 and date.day == 30))
                             else 0 for date in new_data.index]  #good feature
    new_data['flags_day'] = [1 if date.month == 6 and date.day == 14 else 0 for date in new_data.index]
    #********
    # Temperature
    new_data['temperature'] = temperature[:new_data.shape[0]]
    new_data['temperature_wind'] = temperature[:new_data.shape[0]]
    
    # Daily precipitation
    new_data['precipitation'] = dailyWeather['Precip'].values[:new_data.shape[0]]
    new_data['wind_avg'] = daily_weather_h_df['Wind avg'].values[:new_data.shape[0]]
        
    ##############
        
    # Training and predictions
    for data_num in range(6):
        train_num_limit = data.loc[:train_time_limit].shape[0] - data_num
        xgb_train = xgb.DMatrix(new_data.iloc[offset:train_num_limit].drop(['val'], axis=1),\
                                new_data.iloc[offset+data_num+1:train_num_limit+data_num+1].val)
        params={
            'max_depth': 8, 
            'eta': 0.05, 
            'colsample_bytree': 1,
            "min_child_weight": 8, 
            'subsample': 0.8,
            'gamma': 1, # l2-regularization term
            'alpha': 0.01, # l1-regularization term
            'objective': "reg:linear",
            'eval_metric': 'mae',
            'nthread': -1
        }
        evals = [(xgb_train, 'train')]
        bst = xgb.train(params, xgb_train, num_boost_round=200, early_stopping_rounds=10, evals=evals, verbose_eval=False)
        xgb_test = xgb.DMatrix(new_data[train_time_limit:].drop(['val'], axis=1))
        prediction = bst.predict(xgb_test)
        prediction = map(lambda x: max(0,x), prediction) # trips number is non-negative
        difference = denom*np.abs(prediction - \
                        data.loc[pred_start+' 0'+str(0+data_num)+':00:00':pred_end+' '\
                               +str(18+data_num)+':00:00'][region].values)
        error += difference.sum()
        
        indexes = new_data[train_time_limit:].index    
        all_ids.append((pd.Series([str(region)]*indexes.size) + '_' + map(lambda x: x.strftime('%Y-%m-%d'), indexes) + '_' \
                           + map(lambda x: str(x.hour) + '_' + str(data_num+1), indexes)).values)
        all_preds.append(prediction)
    del new_data
    return error, all_preds, all_ids

In [66]:
%%time
# testing elasting net parameter
err, _, _ = count_error_regionGDB(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1075,\
                                denom, temperature, temperature_wind, daily_weather_h_df)
print err
err, _, _ = count_error_regionGDB(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1232,\
                                denom, temperature, temperature_wind, daily_weather_h_df)
print err
err, _, _ = count_error_regionGDB(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 2168,\
                                denom, temperature, temperature_wind, daily_weather_h_df)
print err

0.0971485889424
0.587960340265
0.165405534963
Wall time: 4min 24s


In [67]:
%%time
R = 102
H = 739
Q_may = 0
denom = 1.0/(R*H*6)
Q_may = 0
for region in df.columns:
    print region,
    res, pred, ids = count_error_regionGDB(df.loc[:'2016-05-31 23:00:00'], \
                                        '2016-04-30 23:00:00', '2016-05-31 17:00:00', region,\
                                         denom, temperature, temperature_wind, daily_weather_h_df)
    Q_may += res
print "\n", Q_may

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 
16.0830608085
Wall time: 2h 29min 30s


Качество заметно улучшилось.

Теперь обучим модели на данных до мая 2016 года и посчитаем предсказания. Также еще добавим дневных лагов (это  только улучшит модель, они не были добавлены только для ускорения расчетов).

In [71]:
%%time
R = 102
H = 715
denom = 1.0/(R*H*6)
Q_june = 0
all_ids = []
all_preds = []
for region in df.columns:
    print region,
    res, pred, ids = count_error_regionGDB(df.loc[:'2016-06-30 23:00:00'], \
                                           '2016-05-31 23:00:00', '2016-06-30 17:00:00', region,\
                                           denom, temperature, temperature_wind, daily_weather_h_df,\
                                           '2016-06-01', '2016-06-30', degree=49, K_d=7, K_h=8)
    Q_june += res
    all_ids.append(ids)
    all_preds.append(pred)
pred_df = pd.DataFrame(np.array(all_preds).ravel(), index=np.array(all_ids).ravel(), columns=['y'])
pred_df.index.name = 'id'
pred_df.to_csv("submission.csv", sep=',')

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168Wall time: 2h 40min 13s



In [72]:
print Q_june

15.615440313


Результат улучшился приблизительно на 4.8 пункта.

Ссылка на submission https://inclass.kaggle.com/c/yellowtaxi/leaderboard?submissionId=4288490

Далее приводится код для парсинга погоды с сайта https://www.wunderground.com

In [ ]:
import bs4
import requests
from time import sleep

In [ ]:
data_temperature = []
data_temperature_wind = []
data_wind = []
p = pd.date_range(start='2014-01-01 00:00:00', end='2016-06-30 23:00:00', freq='D')

In [ ]:
for date in p:
    sleep(0.1*np.random.randint(1, 10))
    req = requests.get('https://www.wunderground.com/history/airport/KNYC/{}/{}/{}/'\
          'DailyHistory.html?req_city=New+York&req_state=NY&req_statename=New+York&reqdb.'\
          'zip=10001&reqdb.magic=11&reqdb.wmo=99999&MR=1'.format(date.year, date.month, date.day))
    print date,
    counter = 0
    prev_num = 11
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    p1 = parser.findAll('tr', attrs={'class': "no-metars"})
    for res in p1:
        child = res.findChild('td')
        if not '51' in child.text:
            if not(date.year == 2016 and date.month == 5 and date.day in [13, 14, 15, 16, 18]):
                continue
        a = int(child.text[:child.text.find(':')])
        if a != prev_num+1:
            while a != prev_num+1 or (prev_num == 12 and a!=0):
                if counter == 24:
                    break
                counter += 1
                data_temperature.append(prev)
                data_temperature_wind.append(prev2)
                prev_num += 1
                if prev_num == 12:
                    prev_num = 0
        if counter == 24:
            break
        prev_num = a
        if prev_num == 12:
            prev_num = 0
        child = res.findChild('span', attrs={'class': 'wx-value'})
        counter += 1
        if child:
            data_temperature.append(child.text)
            prev = child.text
        else:
            data_temperature.append(prev)
        child = child.findNext('span', attrs={'class': 'wx-value'})
        if child:
            data_temperature_wind.append(child.text)
            prev2 = child.text
        else:
            data_temperature_wind.append(prev2)
    if prev_num != 11:
        while prev_num != 11:
            if counter == 24:
                break
            counter += 1
            data_temperature.append(prev)
            data_temperature_wind.append(prev2)
            prev_num+=1
    print counter

In [ ]:
with open("hourly_temp_data.pkl", 'wb') as fid:
    pickle.dump(data_temperature, fid)

In [ ]:
with open("hourly_temp_wind_data.pkl", 'wb') as fid:
    pickle.dump(data_temperature_wind, fid)